In [82]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

from scipy.stats import randint, uniform

from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

In [83]:
df = pd.read_csv('Данные для построения модели, задача - Классификация - превышает ли значение IC50 медианное значение выборки.csv')
df

,EState_VSA3,VSA_EState1,PEOE_VSA7,PEOE_VSA9,Chi2n,Chi3n,VSA_EState7,HallKierAlpha,SlogP_VSA5,VSA_EState8,...,EState_VSA6,BCUT2D_CHGLO,EState_VSA2,BCUT2D_MWLOW,PEOE_VSA6,MinAbsPartialCharge,BCUT2D_MRLOW,VSA_EState2,"CC50, mM",Target Feature
0,21.659962,0.000000,74.032366,0.000000,12.058078,10.695991,13.258223,-0.66,105.750639,16.981087,...,11.423370,-2.343082,0.000000,9.700470,54.384066,0.038844,0.193481,10.188192,175.482382,0
1,21.659962,0.000000,97.951860,0.000000,12.648545,11.473090,14.135381,-0.08,105.750639,17.670565,...,0.000000,-2.394690,0.000000,9.689226,54.384066,0.012887,0.120322,0.000000,5.402819,0
2,21.659962,2.517630,74.032366,53.363882,15.671216,13.402236,14.491619,-0.08,105.750639,18.287216,...,0.000000,-2.477203,0.000000,9.681293,41.542423,0.094802,-0.922902,0.000000,161.142320,1
3,21.659962,0.000000,74.032366,0.000000,12.411631,10.945991,14.695439,-0.66,112.171461,17.012013,...,11.423370,-2.342885,0.000000,9.700497,60.804888,0.038844,0.193510,10.194720,107.855654,0
4,10.829981,0.000000,103.003916,11.374773,13.613700,11.833480,8.627311,-2.22,91.194256,14.692318,...,0.000000,-2.342009,10.829981,9.700386,65.807891,0.062897,0.193687,10.301020,139.270991,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
996,11.835812,10.503509,68.114460,24.360600,11.409461,10.058026,7.985276,-1.65,66.219879,8.824371,...,0.000000,-2.677345,52.409521,9.344314,38.841158,0.317890,-0.196385,38.515343,34.999650,0
997,5.917906,10.086396,68.114460,18.945610,12.636189,10.827369,7.521836,-2.18,73.143616,10.378794,...,6.923737,-2.679467,52.347395,9.343622,45.764895,0.327562,-0.187625,51.516353,33.999415,0
998,12.338728,10.305031,79.620167,18.945610,13.248561,11.326709,10.055493,-1.83,72.640700,8.774745,...,11.761885,-2.679527,46.932405,9.343613,45.764895,0.327887,-0.187687,52.527620,33.999458,0
999,19.262465,20.885832,56.278648,51.104983,11.893254,10.158570,2.955837,-2.45,66.219879,7.488627,...,13.847474,-2.652209,30.512100,9.364015,27.192033,0.312509,-0.225309,67.303382,32.999644,0


In [84]:
X, y = df.drop(['Target Feature'], axis=1), df['Target Feature']

In [85]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [86]:
y.value_counts()

Target Feature
0    501
1    500
Name: count, dtype: int64

Классы сбалансированны, значит процесс балансировки классов нам не требуется

In [87]:
class BestClassifierModel:
    """
    Класс для отбора лучшей модели машинного обучения для модели классификации
    X: DataFrame с признаками
    y: Series с целевой переменной

    В данном классе идет подбор 4 разных моделей машинного обучения.
    Для каждой модели производиться подор гипперпараметров (если таковые имеются)
    После получения моделей с лучшими параметрами идет отбор лучшей модели по средству сравнения assessment_criterion с полученными метриками моделей
    """
    def __init__(self, X, y):
        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(X, y, test_size=0.3, random_state=42)
        self.assessment_criterion = {
            'metric_name': 'F1-score',
            'value': 0.7
        }
        self.best_model = None
        self.models = {
            'linear_regression': {
                'model': LogisticRegression(max_iter=1000, random_state=42),
                'best_model': None,
                'best_params': None,
                'params': {
                    'C': [0.01, 0.1, 1, 10],
                    'penalty': ['l1', 'l2'],
                    'solver': ['saga']
                },
                'parameter_selection_method': 'GridSearchCV',
                'metrics': {
                    'Accurac': '',
                    'Precision': '',
                    'Recall': '',
                    'F1-score': '',
                    'ROC-AUC': '',
                },
                'standartization': True,
                'data': {
                    'X_train': self.X_train,
                    'X_test': self.X_test,
                    'y_train': self.y_train,
                    'y_test': self.y_test,
                }
            },
            'elastic_net': {
                'model': KNeighborsClassifier(),
                'best_model': None,
                'best_params': None,
                'parameter_selection_method': 'GridSearchCV',
                'params': {
                    'n_neighbors': range(3, 15),
                    'weights': ['uniform', 'distance'],
                    'p': [1, 2]  # 1 = манхэттенское расстояние, 2 = евклидово расстояние
                },
                'metrics': {
                    'Accurac': '',
                    'Precision': '',
                    'Recall': '',
                    'F1-score': '',
                    'ROC-AUC': '',
                },
                'standartization': True,
                'data': {
                    'X_train': self.X_train,
                    'X_test': self.X_test,
                    'y_train': self.y_train,
                    'y_test': self.y_test,
                }
            },
            'random_forest_regressor': {
                'model': RandomForestClassifier(random_state=42),
                'best_model': None,
                'best_params': None,
                'parameter_selection_method': 'RandomizedSearchCV',
                'params': {
                    'n_estimators': randint(50, 200),
                    'max_depth': [None, 5, 10, 20],
                    'min_samples_split': randint(2, 10),
                    'min_samples_leaf': randint(1, 5)
                },
                'metrics': {
                    'Accurac': '',
                    'Precision': '',
                    'Recall': '',
                    'F1-score': '',
                    'ROC-AUC': '',
                },
                'standartization': False,
                'data': {
                    'X_train': self.X_train,
                    'X_test': self.X_test,
                    'y_train': self.y_train,
                    'y_test': self.y_test,
                }
            },
            'randomized_search_cv_model': {
                'model': SVC(random_state=42, probability=True),  # probability=True для ROC-AUC метрики
                'best_model': None,
                'best_params': None,
                'params': {
                    'C': uniform(0.1, 10),
                    'kernel': ['linear', 'rbf'],
                    'gamma': ['scale', 'auto']
                },
                'parameter_selection_method': 'RandomizedSearchCV',
                'metrics': {
                    'Accurac': '',
                    'Precision': '',
                    'Recall': '',
                    'F1-score': '',
                    'ROC-AUC': '',
                },
                'standartization': True,
                'data': {
                    'X_train': self.X_train,
                    'X_test': self.X_test,
                    'y_train': self.y_train,
                    'y_test': self.y_test,
                    'y_pred': None
                }
            }
        }
        
    
    def fit(self):
        """
        Метод подбора гипперпараметров и инизиализации лучших моделей с подобранными гипперпараметрами для каждой модели
        """
        for model in self.models:
            print(f"Подбор гипперпараметров для модели {str(self.models[model]['model'])} методом {self.models[model]['parameter_selection_method']}")
            
            X, y = self.models[model]['data']['X_train'], self.models[model]['data']['y_train']
            
            if self.models[model]['standartization']:
                print('Перед подобром, необходом провести стандартизацию параметров')
                X = self.X_train_stnd = StandardScaler().fit_transform(X) 
                print('К параметрам применен метод StandardScaler')
                print('Приступаем к подбору гиппепараметров')
            print('Ожидайте...')
            
            if self.models[model]['parameter_selection_method'] == 'GridSearchCV':
                grid = GridSearchCV(self.models[model]['model'], self.models[model]['params'], cv=5, scoring='neg_mean_squared_error')
                grid.fit(X, y)
                self.models[model]['best_params'] = grid.best_params_
                self.models[model]['best_model'] = grid.best_estimator_
            elif self.models[model]['parameter_selection_method'] == 'RandomizedSearchCV':
                search = RandomizedSearchCV(self.models[model]['model'], self.models[model]['params'], n_iter=20, cv=5, scoring='neg_mean_squared_error', random_state=42)
                search.fit(X, y)
                self.models[model]['best_params'] = search.best_params_
                self.models[model]['best_model'] = search.best_estimator_
            print(f"Лучшие параметры для модели {str(self.models[model]['model'])}:", self.models[model]['best_params'])
            print()
                
    def get_best_model(self):
        """
        Метод получения лучшей модели по показателю assessment_criterion
        В данном методе происходит обучение моделей на тестовой выборке и получение метрик на основе полученных предсказаний
        
        По итогу работы модели мы получаем модель, которая имеет лучший показатель по критрию заданному в assessment_criterion или
        уведомление о том, что модель не была выбрана. Тогда необходимо либо изменить метрику, либо изменить значение метрики
        """
        for model in self.models:
            print(f"Обучаем модель {str(self.models[model]['model'])} с лучшими гипперпараметрами для модели:")
            
            X, y = self.models[model]['data']['X_test'], self.models[model]['data']['y_test']
            
            if self.models[model]['standartization']:
                print('Перед обучением, необходом провести стандартизацию параметров')
                X = StandardScaler().fit_transform(X) 
                print('К параметрам применен метод StandardScaler')
            
            print('Приступаем к формированию предсказания на тестовой выборке')
            
            self.models[model]['data']['y_pred'] = self.models[model]['best_model'].predict(X)
            
            self.models[model]['metrics']['Accurac'] = accuracy_score(y, self.models[model]['data']['y_pred'])
            self.models[model]['metrics']['Precision'] = precision_score(y, self.models[model]['data']['y_pred'])
            self.models[model]['metrics']['Recall'] = recall_score(y, self.models[model]['data']['y_pred'])
            self.models[model]['metrics']['F1-score'] = f1_score(y, self.models[model]['data']['y_pred'])
            
            y_proba = self.models[model]['best_model'].predict_proba(X_test)[:, 1]
            self.models[model]['metrics']['ROC-AUC'] = roc_auc_score(y, y_proba)
            print(f"Метрики для модели {str(self.models[model]['model'])}:")
            print(f"Accuracy: {self.models[model]['metrics']['Accurac']:.4f}")
            print(f"Precision: {self.models[model]['metrics']['Precision']:.4f}")
            print(f"Recall: {self.models[model]['metrics']['Recall']:.4f}")
            print(f"F1-score: {self.models[model]['metrics']['F1-score']:.4f}")
            print(f"ROC-AUC: {self.models[model]['metrics']['ROC-AUC']:.4f}")
            print()

            if self.models[model]['metrics'][self.assessment_criterion['metric_name']] > self.assessment_criterion['value']:
                self.assessment_criterion['value'] = self.models[model]['metrics'][self.assessment_criterion['metric_name']]
                self.best_model = self.models[model]
        
        if self.best_model == None:
            print('Модель для формирования прогноза не сформирована')
            print('Измените value или metric_name в assessment_criterion')
        else:
            print('Модель для формирования прогноза сформирована')
            print(f"Лучшая модель по главному критерию оценки {self.assessment_criterion['metric_name']}:")
            print(f"{str(self.best_model['best_model'])}:")
            print('Метрики')
            for k, v in self.best_model['metrics'].items():
                print(f'{k} {v:.4f}')
            
    
    def predict(self, data):
        """
        Метод для формирования прогноза с помощью лучшей модели, которая была получена в методе get_best_model
        """
        result = None
        X = data
        if self.best_model == None:
            print('Модель для формирования прогноза не сформирована, используйте метод get_best_model')
            return result
        else:
            print(f"Формируем предсказание с помощью модели {str(self.best_model['best_model'])}")
            if self.best_model['standartization']:
                print('Перед формированияем прогноза, необходом провести стандартизацию параметров')
                X = StandardScaler().fit_transform(data) 
                print('К параметрам применен метод StandardScaler')
                print('Приступаю формированию прогноза...')
            result = self.best_model['best_model'].predict(X)
            print('Предсказание сформировано')
        return result

In [88]:
best_regression_model = BestClassifierModel(X, y)
best_regression_model.fit()

Подбор гипперпараметров для модели LogisticRegression(max_iter=1000, random_state=42) методом GridSearchCV
Перед подобром, необходом провести стандартизацию параметров
К параметрам применен метод StandardScaler
Приступаем к подбору гиппепараметров
Ожидайте...
Лучшие параметры для модели LogisticRegression(max_iter=1000, random_state=42): {'C': 1, 'penalty': 'l1', 'solver': 'saga'}

Подбор гипперпараметров для модели KNeighborsClassifier() методом GridSearchCV
Перед подобром, необходом провести стандартизацию параметров
К параметрам применен метод StandardScaler
Приступаем к подбору гиппепараметров
Ожидайте...
Лучшие параметры для модели KNeighborsClassifier(): {'n_neighbors': 11, 'p': 2, 'weights': 'distance'}

Подбор гипперпараметров для модели RandomForestClassifier(random_state=42) методом RandomizedSearchCV
Ожидайте...
Лучшие параметры для модели RandomForestClassifier(random_state=42): {'max_depth': 10, 'min_samples_leaf': 3, 'min_samples_split': 7, 'n_estimators': 130}

Подбор ги

In [89]:
best_regression_model.get_best_model()

Обучаем модель LogisticRegression(max_iter=1000, random_state=42) с лучшими гипперпараметрами для модели:
Перед обучением, необходом провести стандартизацию параметров
К параметрам применен метод StandardScaler
Приступаем к формированию предсказания на тестовой выборке
Метрики для модели LogisticRegression(max_iter=1000, random_state=42):
Accuracy: 0.7708
Precision: 0.8211
Recall: 0.6824
F1-score: 0.7454
ROC-AUC: 0.7157

Обучаем модель KNeighborsClassifier() с лучшими гипперпараметрами для модели:
Перед обучением, необходом провести стандартизацию параметров
К параметрам применен метод StandardScaler
Приступаем к формированию предсказания на тестовой выборке
Метрики для модели KNeighborsClassifier():
Accuracy: 0.7110
Precision: 0.7020
Recall: 0.7162
F1-score: 0.7090
ROC-AUC: 0.6852

Обучаем модель RandomForestClassifier(random_state=42) с лучшими гипперпараметрами для модели:
Приступаем к формированию предсказания на тестовой выборке
Метрики для модели RandomForestClassifier(random_sta

g:\PythonProject\Educational_Folder\venv\Lib\site-packages\sklearn\utils\validation.py:2732: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(
g:\PythonProject\Educational_Folder\venv\Lib\site-packages\sklearn\utils\validation.py:2732: UserWarning: X has feature names, but KNeighborsClassifier was fitted without feature names
  warnings.warn(
g:\PythonProject\Educational_Folder\venv\Lib\site-packages\sklearn\utils\validation.py:2732: UserWarning: X has feature names, but SVC was fitted without feature names
  warnings.warn(


In [90]:
best_regression_model.predict(X_test)

Формируем предсказание с помощью модели RandomForestClassifier(max_depth=10, min_samples_leaf=3, min_samples_split=7,
                       n_estimators=130, random_state=42)
Предсказание сформировано


array([1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0,
       0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1,
       0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1,
       1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1,
       0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0,
       1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0,
       1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1,
       0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1,
       1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0,
       1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0,
       0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1,
       0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1]